In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import researchpy as rp
from scipy import stats
from sklearn.cluster import AgglomerativeClustering

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score,precision_score, recall_score, roc_auc_score, log_loss, roc_curve, auc
from sklearn.model_selection import train_test_split, KFold, cross_val_score,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier, GradientBoostingClassifier,BaggingClassifier,StackingClassifier


# Replace 'your_file.csv' with the path to your CSV file.
csv_file = 'in-vehicle-coupon-recommendation.csv'

# Load the CSV file into a pandas DataFrame.
df = pd.read_csv(csv_file)
df = df.drop_duplicates()
print("Shape of dataset after removing duplicates:",df.shape)

# Drop the 'car' , 'toCoupon_GEQ5min' and 'direction_opp' columns
df = df.drop(['car', 'toCoupon_GEQ5min','direction_opp'], axis=1)

weather_col = 'weather'
temp_col = 'temperature'


# Fill missing values with the mode (most common value) of each column
df = df.fillna(df.mode().iloc[0])
# Step 1: Calculate acceptance ratio for each occupation
acceptance_ratio = df.groupby('occupation')['Y'].mean().sort_values()

In [ ]:
# Step 2: Bin occupations based on acceptance ratio
# Using quantiles as dynamic boundaries. Adjust according to your needs.
bins = [
    acceptance_ratio.min(),
    acceptance_ratio.quantile(0.2),
    acceptance_ratio.quantile(0.4),
    acceptance_ratio.quantile(0.6),
    acceptance_ratio.quantile(0.8),
    acceptance_ratio.max()
]

bin_labels = ['low', 'medium_low', 'medium', 'medium_high', 'high']

# Assign bin labels
occupation_bins = pd.cut(acceptance_ratio, bins=bins, labels=bin_labels, include_lowest=True)

# Step 3: Map original occupation to occupation_class
df['occupation'] = df['occupation'].map(occupation_bins.to_dict())


# Define a function to combine the features into 'toCoupon'
def combine_features(row):
    if row['toCoupon_GEQ15min'] == 0:  # driving distance <= 15 min
        return 0
    elif row['toCoupon_GEQ25min'] == 0 :  # driving distance > 15 min and <= 25 min
        return 1
    else:  # driving distance > 25 min
        return 2

# Apply the function to the dataframe
df['toCoupon'] = df.apply(combine_features, axis=1)

# Optionally, drop the original features
df = df.drop(['toCoupon_GEQ15min', 'toCoupon_GEQ25min'], axis=1)
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=df, x='toCoupon', hue='Y', palette='viridis', edgecolor='black', linewidth=0.7)


# Feature Extraction for 'passenger_destination' from 'time' and 'destination'
df['time_destination'] = df['time'].astype(str) + "_" + df['destination'].astype(str)

# Feature Extraction for 'marital_hasChildren' from 'maritalStatus' and 'has_children'
df['marital_hasChildren'] = df['maritalStatus'].astype(str) + "_" + df['has_children'].astype(str)

# Feature Extraction for 'temperature_weather' from 'temperature' and 'weather'
df['temperature_weather'] = df['temperature'].astype(str) + "_" + df['weather'].astype(str)


df = df.drop(['time', 'destination', 'maritalStatus', 'has_children', 'temperature', 'weather'], axis=1)



# Define order for the ordinal variables

age_order = {'below21': 0, '21': 1, '26': 2, '31': 3, '36': 4, '41': 5, '46': 6, '50plus': 7}
education_order = {'Some High School': 0, 'High School Graduate': 1, 'Some college - no degree': 2, 'Associates degree': 3, 'Bachelors degree': 4, 'Graduate degree (Masters or Doctorate)': 5}
income_order = {'Less than $12500': 0, '$12500 - $24999': 1, '$25000 - $37499': 2, '$37500 - $49999': 3, '$50000 - $62499': 4, '$62500 - $74999': 5, '$75000 - $87499': 6, '$87500 - $99999': 7, '$100000 or More': 8}
frequency_order = {'never': 0, 'less1': 1, '1~3': 2, '4~8': 3, 'gt8': 4}
occupation_order= { 'medium_low':1, 'high':4, 'medium_high':3, 'low' :0,'medium':2}

# Replace the values based on the order
df['age'] = df['age'].replace(age_order)
df['education'] = df['education'].replace(education_order)
df['income'] = df['income'].replace(income_order)
df['occupation']=df['occupation'].replace(occupation_order)

# Encoding frequency-like features
for col in ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']:
    df[col] = df[col].replace(frequency_order)


# 1. One-Hot Encoding
onehot_cols = ['passanger', 'coupon', 'marital_hasChildren', 'temperature_weather', 'time_destination']
encoder = OneHotEncoder(sparse_output=False, drop='first')  
encoded_cols = pd.DataFrame(encoder.fit_transform(df[onehot_cols]))

# Reset indices to ensure alignment when concatenating
encoded_cols.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# Use appropriate column names for one-hot encoded columns
encoded_cols.columns = encoder.get_feature_names_out(onehot_cols)

# Concatenate the original dataframe and the one-hot encoded columns
df = pd.concat([df, encoded_cols], axis=1)

# Drop the original columns that were one-hot encoded
df.drop(onehot_cols, axis=1, inplace=True)

# 2. Binary Encoding
df['expiration'] = df['expiration'].map({'2h': 0, '1d': 1})
# Note: 'Y' and 'direction_same' are already binary, no encoding needed

# 3. Label Encoding
label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])  # 0 for Female and 1 for Male

# Dataframe modelling

In [ ]:
# Define features (X) and target (y)
X = df.drop("Y", axis=1)
y = df["Y"]
X = X.rename(columns={'coupon_Restaurant(<20)': 'coupon_Restaurant(20)'})

#save X as csv
#X.to_csv('X.csv', index=False)

# Split data into 75% train+validation and 25% test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize a k-fold cross-validator (e.g., 5 folds)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Dimensionality Reduction

In [ ]:
# Normalize features
scaler = StandardScaler()
X_standerdized= scaler.fit_transform(X_train_val)

### PCA (Principal Component Analysis)

In [ ]:
from sklearn.decomposition import PCA

# Perform PCA to reduce dimensionality to 2 dimensions
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_standerdized)
df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

### t-Distributed Stochastic Neighbor Embedding (t-SNE)

In [ ]:
from sklearn.manifold import TSNE

# Perform t-SNE to reduce dimensionality to 2 dimensions
tsne = TSNE(n_components=2, random_state=42)
tsne_components = tsne.fit_transform(X_standerdized)
df_tsne = pd.DataFrame(data=tsne_components, columns=['t-SNE1', 't-SNE2'])

# Clustering

### K-Means

In [ ]:
from sklearn.cluster import KMeans

# Apply K-Means clustering (assuming 3 clusters for illustration)
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X_standerdized)

# Add cluster labels to the PCA dataframe
df_pca['cluster'] = clusters

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='cluster', palette='viridis')
plt.title('K-Means Clustering with 3 Clusters')
plt.show()

### Agglomerative

In [ ]:
# Perform hierarchical clustering
cluster = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage='ward')
df_train_clusters = pd.DataFrame({'Cluster': cluster.fit_predict(X_standerdized)}, index=X_train_val.index)

# Visualize the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=df_train_clusters.join(df_pca), palette='viridis')
plt.title('Hierarchical Clustering - Train Set')
plt.show()